In [ ]:
# !pip install openai python-aiconfig

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.6/381.6 kB 24.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.9/267.9 kB 27.2 MB/s

In [ ]:
# import openai
# import aiconfig
# from google.colab import userdata
# openai.api_key = userdata.get('OPENAI_API_KEY')


/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_parsers" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [8]:
# Non-Colab
import json
import openai
openai.api_key = open("C:\\Users\\2time\\Projects\\secrets\\openai_api_key.txt", 'r').read()

## Function Calling

In [20]:
from pydantic import create_model
from typing import Callable, List
import inspect, json
from inspect import Parameter
def sums(a:int, b:int=1):
    "Adds a + b"
    return a + b
def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [29]:
def build_function_call_spec(functions: List[Callable]):
    spec = json.load(open("function_calling_stub_aiconfig.json", 'r'))
    spec["prompts"][0]["metadata"]["model"]["settings"]["functions"] = [schema(f) for f in functions]
    return spec



In [30]:
import copy
import json
import uuid
from aiconfig import AIConfigRuntime
from aiconfig import Prompt
from aiconfig import PromptInput
from aiconfig.model_parser import InferenceOptions
from pprint import pprint


async def function_calling(function_list):
    function_call_spec = build_function_call_spec(function_list)
    config = AIConfigRuntime.create(**function_call_spec)

    params = {
        "book": "Where the Crawdads Sing",
    }

    completionParams = await config.resolve("recommendBook", params)

    print("completionParams=", end=" ")
    pprint(completionParams)

    messages = completionParams["messages"]
    print(messages[0])
    print(messages[1])
    print()

    promptToRun = "recommendBook"

    inference_options = InferenceOptions(stream=True)

    while True:
        model_output = await config.run(promptToRun, params, inference_options)

        output = model_output[0] if isinstance(model_output, list) else model_output

        if output.output_type == "error":
            print(f"Error during inference: {output.ename}: {output.evalue}")
            return config

        message = output.data

        # If there is no function call, we're done and can exit this loop
        if not message.get("function_call", None):
            return config

        # If there is a function call, we generate a new message with the role 'function'.

        result = await callFunction(message.get("function_call"))

        new_message = {
            "role": "function",
            "name": message["function_call"]["name"],
            "content": json.dumps(result),
        }

        promptToRun = f"functionCallResult-{uuid.uuid4()}"

        existing_prompt = config.get_prompt("recommendBook")

        new_prompt = copy.deepcopy(existing_prompt)
        new_prompt.name = promptToRun
        new_prompt.input = PromptInput(**new_message)
        new_prompt.outputs = []

        config.add_prompt(new_prompt.name, new_prompt)

        print(f"{new_message}\n")


db = [
    {
        "id": "a1",
        "name": "To Kill a Mockingbird",
        "genre": "historical",
        "description": 'Compassionate, dramatic, and deeply moving, "To Kill A Mockingbird" takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a masterpiece of American literature.',
    },
    {
        "id": "a2",
        "name": "All the Light We Cannot See",
        "genre": "historical",
        "description": "In a mining town in Germany, Werner Pfennig, an orphan, grows up with his younger sister, enchanted by a crude radio they find that brings them news and stories from places they have never seen or imagined. Werner becomes an expert at building and fixing these crucial new instruments and is enlisted to use his talent to track down the resistance. Deftly interweaving the lives of Marie-Laure and Werner, Doerr illuminates the ways, against all odds, people try to be good to one another.",
    },
    {
        "id": "a3",
        "name": "Where the Crawdads Sing",
        "genre": "historical",
        "description": """For years, rumors of the “Marsh Girl” haunted Barkley Cove, a quiet fishing village. Kya Clark is barefoot and wild; unfit for polite society. So in late 1969, when the popular Chase Andrews is found dead, locals immediately suspect her.

But Kya is not what they say. A born naturalist with just one day of school, she takes life's lessons from the land, learning the real ways of the world from the dishonest signals of fireflies. But while she has the skills to live in solitude forever, the time comes when she yearns to be touched and loved. Drawn to two young men from town, who are each intrigued by her wild beauty, Kya opens herself to a new and startling world—until the unthinkable happens.""",
    },
]


async def list_items(genre: str):
    """list queries books by genre, and returns a list of names of books"""
    # filter
    items = [item for item in db if item["genre"] == genre]
    # map
    return [{"name": item["name"], "id": item["id"]} for item in items]


async def search(name: str):
    """search queries books by their name and returns a list of book names and their ids"""
    # filter
    items = [item for item in db if name in item["name"]]
    # map
    return [{"name": item["name"], "id": item["id"]} for item in items]


async def get(id: str):
    """get returns a book's detailed information based on the id of the book. Note that this does not accept names, and only IDs, which you can get by using search."""
    return [item for item in db if item["id"] == id][0] or None


async def callFunction(function_call):
    args = function_call.get("arguments", None)
    args = json.loads(args) if args else None

    if not args:
        raise Exception("No arguments found")

    match function_call.get("name"):
        case "list_items":
            return await list_items(args["genre"])
        case "search":
            return await search(args["name"])
        case "get":
            return await get(args["id"])



In [31]:

partial_spec = json.load(open("function_calling_stub_aiconfig.json", 'r'))
partial_spec["prompts"][0]["metadata"]["model"]["settings"]["functions"]

[]

In [32]:
function_call_spec = build_function_call_spec([list_items, search, get])
function_call_spec

C:\Users\2time\AppData\Local\Temp\ipykernel_14432\107065073.py:11: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


{'name': 'function-call-demo',
 'description': 'this is a demo AIConfig to show function calling using OpenAI',
 'schema_version': 'latest',
 'metadata': {},
 'prompts': [{'name': 'recommendBook',
   'input': {'role': 'user',
    'content': 'I really enjoyed reading {{book}}, could you recommend me a book that is similar and tell me why?'},
   'metadata': {'model': {'name': 'gpt-3.5-turbo',
     'settings': {'model': 'gpt-3.5-turbo',
      'functions': [{'name': 'list_items',
        'description': 'list queries books by genre, and returns a list of names of books',
        'parameters': {'properties': {'genre': {'title': 'Genre',
           'type': 'string'}},
         'required': ['genre'],
         'title': 'Input for `list_items`',
         'type': 'object'}},
       {'name': 'search',
        'description': 'search queries books by their name and returns a list of book names and their ids',
        'parameters': {'properties': {'name': {'title': 'Name',
           'type': 'string'

In [34]:
book_search = await function_calling([list_items, search, get])

C:\Users\2time\AppData\Local\Temp\ipykernel_14432\107065073.py:11: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  s = create_model(f'Input for `{f.__name__}`', **kw).schema()


completionParams= {'functions': [{'description': 'list queries books by genre, and returns a '
                               'list of names of books',
                'name': 'list_items',
                'parameters': {'properties': {'genre': {'title': 'Genre',
                                                        'type': 'string'}},
                               'required': ['genre'],
                               'title': 'Input for `list_items`',
                               'type': 'object'}},
               {'description': 'search queries books by their name and returns '
                               'a list of book names and their ids',
                'name': 'search',
                'parameters': {'properties': {'name': {'title': 'Name',
                                                       'type': 'string'}},
                               'required': ['name'],
                               'title': 'Input for `search`',
                               'type': 'obj

In [35]:
book_search.save("book_search.json")